# Lab 1 

- It is recommended to **go through this file with a partner**. 
- Ensure to **ask** if anything is not clear - first your partner, then a lab helper.
- You want to first go through the accompanying code of the lectures.

## Ex. 1 - Different Phases

You are given the following SIMP program: 
```
VAR n; 
n = 5; 
WHILE n > 0 DO 
    BEGIN
    PRINT n;
    n := n - 1 
    END
```

1. What does its lexical syntax look like on paper? 
2. What does its phrasal syntax look like on paper? 
3. What does its abstract syntax look like? (Both on paper and in OCaml!)
4. What does a corresponding assembler program look like on paper? (You can use ARM instructions or the MIPS instruction set: https://www.dsi.unive.it/~gasparetto/materials/MIPS_Instruction_Set.pdf - in the lecture, we will use the latter.
You can test your program here: https://cpulator.01xz.net/?sys=mipsr5)

In [16]:
type op = Plus | Minus | Mult | Div 

type exp = Id of string | Numb of int | Op of exp * op * exp 

type cond = Eq | Neq | Lte | Lt | Gte | Gt 
type condexp = Cop of exp * cond * exp
                                          
type cmd = Asgn of string * exp 
         | Ite of condexp * cmd * cmd | If of condexp * cmd 
         | While of condexp * cmd
         | Begin of program 
         | Print of exp 
         
and program = Program of string list * cmd list

let p : program = (* TODO *)

Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads



type op = Plus | Minus | Mult | Div


type exp = Id of string | Numb of int | Op of exp * op * exp


type cond = Eq | Neq | Lte | Lt | Gte | Gt


type condexp = Cop of exp * cond * exp


type cmd =
    Asgn of string * exp
  | Ite of condexp * cmd * cmd
  | If of condexp * cmd
  | While of condexp * cmd
  | Begin of program
  | Print of exp
and program = Program of string list * cmd list


val p : program = Program ([], [])


## Ex. 2 - Error Analysis 

When do the following errors appear - lexical analysis, syntactic analysis, semantic analysis, or during the interpreter/execution? Static or dynamic?

Program 1 in SIMP: 
```
VAR 1var;
1var = 3
```

Program 2 in SIMP: 
```
VAR x
```

Program 3 in SIMP: 
```
VAR x;
x = 2 + 3;
```

Program 4 in C: 
```
char *cp = 'x';
```

Program 5 in Java: 
```
class X {
  void increment() { ... }
}

X x = new X;
x.decrement();
```

Program 6 in Python: 
```
class X:
  def increment(self):
    ...

x = X()
x.decrement()
```

## Ex. 3 - Interpretation 

Below is the full code for the interpreter you saw in class for SIMP:

In [19]:
type op = Plus | Minus | Mult | Div 

type exp = Id of string | Numb of int | Op of exp * op * exp 

type cond = Eq | Neq | Lte | Lt | Gte | Gt 
type condexp = Cop of exp * cond * exp
                                          
type cmd = Asgn of string * exp 
         | Ite of condexp * cmd * cmd | If of condexp * cmd 
         | While of condexp * cmd
         | Begin of program 
         | Print of exp 
         
and program = Program of string list * cmd list

(* This will define maps with strings as key *)
module Env = Map.Make(String)

(* Env.empty denotes the empty environment. 
We can add elements to an environment via Env.add.
This is the environment which only binds “a” to 3. *)
let example_env = Env.add "a" 3 Env.empty;;

(* We can look up elements in an environment via Env.find.
Env.find throws an exception if the key does not exist.*)
Env.find "a" example_env;;

exception RuntimeError of string

let rec eval (e : exp) env : int = match e with 
    | Id x -> Env.find x env 
    | Numb n -> n 
    | Op (e1, Plus, e2) -> eval e1 env + eval e2 env
    | Op (e1, Minus, e2) -> eval e1 env - eval e2 env
    | Op (e1, Mult, e2) -> eval e1 env * eval e2 env
    | Op (e1, Div, e2) ->  (match (eval e2 env) with 
                          | 0 -> raise (RuntimeError "Divison by 0")
                          | _ -> eval e1 env / eval e2 env)
                          
let eval_condexp (e : condexp) env : int = match e with                          
    | Cop (e1, Eq, e2) -> if (eval e1 env = eval e2 env) then 1 else 0
    | Cop (e1, Neq, e2) -> if (eval e1 env = eval e2 env) then 0 else 1
    | Cop (e1, Lte, e2) -> if (eval e1 env <= eval e2 env) then 1 else 0
    | Cop (e1, Lt, e2) -> if (eval e1 env < eval e2 env) then 1 else 0
    | Cop (e1, Gte, e2) -> if (eval e1 env >= eval e2 env) then 1 else 0
    | Cop (e1, Gt, e2) -> if (eval e1 env > eval e2 env) then 1 else 0

let rec eval_cmd (c : cmd) env  = match c with 
    | Asgn (x, e) -> Env.add x (eval e env) env
    | If (e, c) -> (match (eval_condexp e env) with 
                        | 1 -> eval_cmd c env 
                        | 0 -> env
                        | _ -> raise (RuntimeError "Error in condition of If")
                        )
    | Ite (e, c1, c2) -> (match (eval_condexp e env) with 
                        | 1 -> eval_cmd c1 env 
                        | 0 -> eval_cmd c2 env
                        | _ -> raise (RuntimeError "Error in condition of If"))
    | While (e, c) -> (match (eval_condexp e env) with 
                        | 1 -> eval_cmd (While (e, c)) (eval_cmd c env)
                        | _ -> env)
    | Begin p -> let _ = eval_program p env in env
    | Print e -> let _ = print_endline ("OUTPUT:" ^ string_of_int (eval e env)) in env
    
and eval_program p env = match p with 
    | Program (xs, cmds) -> eval_commands cmds Env.empty

and eval_commands (cs : cmd list) env = match cs with 
    | [] -> env
    | c :: cs -> eval_commands cs (eval_cmd c env)

type op = Plus | Minus | Mult | Div


type exp = Id of string | Numb of int | Op of exp * op * exp


type cond = Eq | Neq | Lte | Lt | Gte | Gt


type condexp = Cop of exp * cond * exp


type cmd =
    Asgn of string * exp
  | Ite of condexp * cmd * cmd
  | If of condexp * cmd
  | While of condexp * cmd
  | Begin of program
  | Print of exp
and program = Program of string list * cmd list


...

- : int = 3


exception RuntimeError of string


val eval : exp -> int Env.t -> int = <fun>


val eval_condexp : condexp -> int Env.t -> int = <fun>


val eval_cmd : cmd -> int Env.t -> int Env.t = <fun>
val eval_program : program -> int Env.t -> int Env.t = <fun>
val eval_commands : cmd list -> int Env.t -> int Env.t = <fun>


1. Extend expressions and the interpreter with a unary negation operator for integers, ``Neg``. 
Write an example of code and test that it evaluates correctly. 

In [ ]:
(* Write an example expression *)

let e = (* TODO *) ;;

eval e;;

error: compile_error

2. Extend commands with a ``DoWhile`` loop. 

See the following explanation of Wikipedia: 
*The do while construct consists of a process symbol and a condition. First the code within the block is executed. Then the condition is evaluated. If the condition is true the code within the block is executed again. This repeats until the condition becomes false.*

Write an example of code and test that it evaluates correctly.

In [ ]:
(* Write an example program *)

let p = (* TODO *) ;;

eval_program p Env.empty;;

3. Write a domain-specific language to manipulate Strings. 

You want to be able to: 
- Define variables of type String. 
- Concatenate Strings.
- Print Strings.

a.) Define a datatype ``string_exp`` of string expressions. 
   Your data type should contain: 
   - identifiers which represent strings.
   - the empty string.
   - a possibility to concatenate Strings.

b.) Define a datatype ``string_stm`` of statements manipulating string variables. 
    They should contain:
    - assignments 
    - the possibility to concatenate statements 
    
c.) Define an interpreter for 1.) expressions, and 2.) statements.

d.) (Optional - challenge) Build the possibility of Strings into ``Simp``. 

In [ ]:
(* Your code comes here *)



## Ex. 4 - Challenge

Extend SIMP with unary function definitions: 
```

type cmd = ... | FunctionDef of string (* function name *) 
                 * string (* arg name *) 
                 * cmd
```

and epxressions with a way to call a function: 
```

type exp = ... | App of string (* function *) 
                        * exp (* argument *)
```

Define an interpreter for this extended language. 

In [ ]:
(* Your code comes here *)

